In [9]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import itertools

def ruwt_scraper(date, excitement, rating):
    URL = f'https://areyouwatchingthis.com/scores/{date}'
    
    page = requests.get(URL)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    ruwt_class = soup.find_all('li', attrs={'class': excitement})
    
    data = []
    data_dict = {}
    for i in ruwt_class:
        try:
            score1 = int(re.sub("\D", "", i.find_all('span', class_='score')[0].text))
        except IndexError:
            score1 = 0
        try:
            score2 = int(re.sub("\D", "", i.find_all('span', class_='score')[1].text))
        except IndexError:
            score2 = 0
        try:
            team1 = i.find_all('a', class_='team')[0].text.strip()
        except IndexError:
            team1 = i.find('a', class_='field').text.strip()
        try:
            team2 = i.find_all('a', class_='team')[1].text.strip()
        except IndexError:
            team2 = ''
        data_dict = {
            'sport' : i.find('a', class_='sport').text.strip(),
            'team1' : team1,
            'team2' : team2,
            'score1' : score1,
            'score2' : score2,
            'score_diff' : abs(score1 - score2),
            'score_sum' : score1 + score2,
            'rating' : rating
        }
        data.append(data_dict)

    df = pd.DataFrame(data)
    
    return df

def this_date_in_ruwt(date):
    df_severe = ruwt_scraper(date, 'severe', 1)
    df_high = ruwt_scraper(date, 'high', 1)
    df_elevated = ruwt_scraper(date, 'elevated', 0)
    df_guarded = ruwt_scraper(date, 'guarded', 0)
    pdList = [df_severe, df_high, df_elevated, df_guarded]
    df = pd.concat(pdList)
    
    df.to_csv(f'/Users/blakeduncan/Downloads/RUWT/{date}.csv', index=False)
    
    return df

In [10]:
date = [ 20151231,
 20151230,
 20151229,
 20151228,
 20151227,
 20151226,
 20151225,
 20151224,
 20151223,
 20151222,
 20151221,
 20151220,
 20151219,
 20151218,
 20151217,
 20151216,
 20151215,
 20151214,
 20151213,
 20151212,
 20151211,
 20151210,
 20151209,
 20151208,
 20151207,
 20151206,
 20151205,
 20151204,
 20151203,
 20151202,
 20151201,
 20151130,
 20151129,
 20151128,
 20151127,
 20151126,
 20151125,
 20151124,
 20151123,
 20151122,
 20151121,
 20151120,
 20151119,
 20151118,
 20151117,
 20151116,
 20151115,
 20151114,
 20151113,
 20151112,
 20151111,
 20151110,
 20151109,
 20151108,
 20151107,
 20151106,
 20151105,
 20151104,
 20151103,
 20151102,
 20151101,
 20151031,
 20151030,
 20151029,
 20151028,
 20151027,
 20151026,
 20151025,
 20151024,
 20151023,
 20151022,
 20151021,
 20151020,
 20151019,
 20151018,
 20151017,
 20151016,
 20151015,
 20151014,
 20151013,
 20151012,
 20151011,
 20151010,
 20151009,
 20151008,
 20151007,
 20151006,
 20151005,
 20151004,
 20151003,
 20151002,
 20151001,
 20150930,
 20150929,
 20150928,
 20150927,
 20150926,
 20150925,
 20150924,
 20150923,
 20150922,
 20150921,
 20150920,
 20150919,
 20150918,
 20150917,
 20150916,
 20150915,
 20150914,
 20150913,
 20150912,
 20150911,
 20150910,
 20150909,
 20150908,
 20150907,
 20150906,
 20150905,
 20150904,
 20150903,
 20150902,
 20150901,
 20150831,
 20150830,
 20150829,
 20150828,
 20150827,
 20150826,
 20150825,
 20150824,
 20150823,
 20150822,
 20150821,
 20150820,
 20150819,
 20150818,
 20150817,
 20150816,
 20150815,
 20150814,
 20150813,
 20150812,
 20150811,
 20150810,
 20150809,
 20150808,
 20150807,
 20150806,
 20150805,
 20150804,
 20150803,
 20150802,
 20150801,
 20150731,
 20150730,
 20150729,
 20150728,
 20150727,
 20150726,
 20150725,
 20150724,
 20150723,
 20150722,
 20150721,
 20150720,
 20150719,
 20150718,
 20150717,
 20150716,
 20150715,
 20150714,
 20150713,
 20150712,
 20150711,
 20150710,
 20150709,
 20150708,
 20150707,
 20150706,
 20150705,
 20150704,
 20150703,
 20150702,
 20150701,
 20150630,
 20150629,
 20150628,
 20150627,
 20150626,
 20150625,
 20150624,
 20150623,
 20150622,
 20150621,
 20150620,
 20150619,
 20150618,
 20150617,
 20150616,
 20150615,
 20150614,
 20150613,
 20150612,
 20150611,
 20150610,
 20150609,
 20150608,
 20150607,
 20150606,
 20150605,
 20150604,
 20150603,
 20150602,
 20150601,
 20150531,
 20150530,
 20150529,
 20150528,
 20150527,
 20150526,
 20150525,
 20150524,
 20150523,
 20150522,
 20150521,
 20150520,
 20150519,
 20150518,
 20150517,
 20150516,
 20150515,
 20150514,
 20150513,
 20150512,
 20150511,
 20150510,
 20150509,
 20150508,
 20150507,
 20150506,
 20150505,
 20150504,
 20150503,
 20150502,
 20150501,
 20150430,
 20150429,
 20150428,
 20150427,
 20150426,
 20150425,
 20150424,
 20150423,
 20150422,
 20150421,
 20150420,
 20150419,
 20150418,
 20150417,
 20150416,
 20150415,
 20150414,
 20150413,
 20150412,
 20150411,
 20150410,
 20150409,
 20150408,
 20150407,
 20150406,
 20150405,
 20150404,
 20150403,
 20150402,
 20150401,
 20150331,
 20150330,
 20150329,
 20150328,
 20150327,
 20150326,
 20150325,
 20150324,
 20150323,
 20150322,
 20150321,
 20150320,
 20150319,
 20150318,
 20150317,
 20150316,
 20150315,
 20150314,
 20150313,
 20150312,
 20150311,
 20150310,
 20150309,
 20150308,
 20150307,
 20150306,
 20150305,
 20150304,
 20150303,
 20150302,
 20150301,
 20150228,
 20150227,
 20150226,
 20150225,
 20150224,
 20150223,
 20150222,
 20150221,
 20150220,
 20150219,
 20150218,
 20150217,
 20150216,
 20150215,
 20150214,
 20150213,
 20150212,
 20150211,
 20150210,
 20150209,
 20150208,
 20150207,
 20150206,
 20150205,
 20150204,
 20150203,
 20150202,
 20150201,
 20150131,
 20150130,
 20150129,
 20150128,
 20150127,
 20150126,
 20150125,
 20150124,
 20150123,
 20150122,
 20150121,
 20150120,
 20150119,
 20150118,
 20150117,
 20150116,
 20150115,
 20150114,
 20150113,
 20150112,
 20150111,
 20150110,
 20150109,
 20150108,
 20150107,
 20150106,
 20150105,
 20150104,
 20150103,
 20150102,
 20150101,
 20141231,
 20141230,
 20141229,
 20141228,
 20141227,
 20141226,
 20141225,
 20141224,
 20141223,
 20141222,
 20141221,
 20141220,
 20141219,
 20141218,
 20141217,
 20141216,
 20141215,
 20141214,
 20141213,
 20141212,
 20141211,
 20141210,
 20141209,
 20141208,
 20141207,
 20141206,
 20141205,
 20141204,
 20141203,
 20141202,
 20141201,
 20141130,
 20141129,
 20141128,
 20141127,
 20141126,
 20141125,
 20141124,
 20141123,
 20141122,
 20141121,
 20141120,
 20141119,
 20141118,
 20141117,
 20141116,
 20141115,
 20141114,
 20141113,
 20141112,
 20141111,
 20141110,
 20141109,
 20141108,
 20141107,
 20141106,
 20141105,
 20141104,
 20141103,
 20141102,
 20141101,
 20141031,
 20141030,
 20141029,
 20141028,
 20141027,
 20141026,
 20141025,
 20141024,
 20141023,
 20141022,
 20141021,
 20141020,
 20141019,
 20141018,
 20141017,
 20141016,
 20141015,
 20141014,
 20141013,
 20141012,
 20141011,
 20141010,
 20141009,
 20141008,
 20141007,
 20141006,
 20141005,
 20141004,
 20141003,
 20141002,
 20141001,
 20140930,
 20140929,
 20140928,
 20140927,
 20140926,
 20140925,
 20140924,
 20140923,
 20140922,
 20140921,
 20140920,
 20140919,
 20140918,
 20140917,
 20140916,
 20140915,
 20140914,
 20140913,
 20140912,
 20140911,
 20140910,
 20140909,
 20140908,
 20140907,
 20140906,
 20140905,
 20140904,
 20140903,
 20140902,
 20140901,
 20140831,
 20140830,
 20140829,
 20140828,
 20140827,
 20140826,
 20140825,
 20140824,
 20140823,
 20140822,
 20140821,
 20140820,
 20140819,
 20140818,
 20140817,
 20140816,
 20140815,
 20140814,
 20140813,
 20140812,
 20140811,
 20140810,
 20140809,
 20140808,
 20140807,
 20140806,
 20140805,
 20140804,
 20140803,
 20140802,
 20140801,
 20140731,
 20140730,
 20140729,
 20140728,
 20140727,
 20140726,
 20140725,
 20140724,
 20140723,
 20140722,
 20140721,
 20140720,
 20140719,
 20140718,
 20140717,
 20140716,
 20140715,
 20140714,
 20140713,
 20140712,
 20140711,
 20140710,
 20140709,
 20140708,
 20140707,
 20140706,
 20140705,
 20140704,
 20140703,
 20140702,
 20140701,
 20140630,
 20140629,
 20140628,
 20140627,
 20140626,
 20140625,
 20140624,
 20140623,
 20140622,
 20140621,
 20140620,
 20140619,
 20140618,
 20140617,
 20140616,
 20140615,
 20140614,
 20140613,
 20140612,
 20140611,
 20140610,
 20140609,
 20140608,
 20140607,
 20140606,
 20140605,
 20140604,
 20140603,
 20140602,
 20140601,
 20140531,
 20140530,
 20140529,
 20140528,
 20140527,
 20140526,
 20140525,
 20140524,
 20140523,
 20140522,
 20140521,
 20140520,
 20140519,
 20140518,
 20140517,
 20140516,
 20140515,
 20140514,
 20140513,
 20140512,
 20140511,
 20140510,
 20140509,
 20140508,
 20140507,
 20140506,
 20140505,
 20140504,
 20140503,
 20140502,
 20140501,
 20140430,
 20140429,
 20140428,
 20140427,
 20140426,
 20140425,
 20140424,
 20140423,
 20140422,
 20140421,
 20140420,
 20140419,
 20140418,
 20140417,
 20140416,
 20140415,
 20140414,
 20140413,
 20140412,
 20140411,
 20140410,
 20140409,
 20140408,
 20140407,
 20140406,
 20140405,
 20140404,
 20140403,
 20140402,
 20140401,
 20140331,
 20140330,
 20140329,
 20140328,
 20140327,
 20140326,
 20140325,
 20140324,
 20140323,
 20140322,
 20140321,
 20140320,
 20140319,
 20140318,
 20140317,
 20140316,
 20140315,
 20140314,
 20140313,
 20140312,
 20140311,
 20140310,
 20140309,
 20140308,
 20140307,
 20140306,
 20140305,
 20140304,
 20140303,
 20140302,
 20140301,
 20140228,
 20140227,
 20140226,
 20140225,
 20140224,
 20140223,
 20140222,
 20140221,
 20140220,
 20140219,
 20140218,
 20140217,
 20140216,
 20140215,
 20140214,
 20140213,
 20140212,
 20140211,
 20140210,
 20140209,
 20140208,
 20140207,
 20140206,
 20140205,
 20140204,
 20140203,
 20140202,
 20140201,
 20140131,
 20140130,
 20140129,
 20140128,
 20140127,
 20140126,
 20140125,
 20140124,
 20140123,
 20140122,
 20140121,
 20140120,
 20140119,
 20140118,
 20140117,
 20140116,
 20140115,
 20140114,
 20140113,
 20140112,
 20140111,
 20140110,
 20140109,
 20140108,
 20140107,
 20140106,
 20140105,
 20140104,
 20140103,
 20140102,
 20140101]
for i in date:
    this_date_in_ruwt(i)

In [11]:
import pandas as pd
import glob

path = r'/Users/blakeduncan/Downloads/RUWT' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [12]:
frame.to_csv(f'/Users/blakeduncan/Downloads/RUWT.csv', index=False)

In [13]:
frame = frame['sport'].value_counts()
print(frame)

NCAAB             35614
SOCCER            21624
NCAABW            20921
MLB               17162
NCAAF             11544
NHL                8668
NBA                8435
MMA                2967
TENNIS             2249
NFL                1992
AHL                1747
NCAAH              1708
ECHL               1526
WNBA               1391
AUSSIEFOOTBALL      926
RACING              838
NBDL                790
RUGBYUNION          788
RUGBYLEAGUE         708
CRICKET             618
ESPORTS             603
BOXING              435
GOLF                303
NETBALL             302
NBL                 232
CHL                 225
AFL                 213
CFL                 188
LACROSSE            120
OLYMPICS              2
Name: sport, dtype: int64
